# Analyze the data

In [7]:
import os
import pickle
import numpy
from music21 import note, chord, instrument, stream, duration
import time

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
from models.RNNAttention import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network
from models.RNNAttention import create_network, sample_with_temp
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd


In [8]:
#load data and mappings

melody_lengths = [12,16,24]
data_dir = 'data/'

notes_in_dict = {}
notes_out_dict = {}
durations_in_dict = {}
durations_out_dict = {}
for l in melody_lengths:
    notes_in_dict[l] = np.load(data_dir + 'SEQ_' +str(l) + '/notes_in_' + str(l) + '.npy')
    notes_out_dict[l] = np.load(data_dir + 'SEQ_' +str(l) + '/notes_out_' + str(l) + '.npy')
    durations_in_dict[l] = np.load(data_dir + 'SEQ_' +str(l) +'/durations_in_' + str(l) + '.npy')
    durations_out_dict[l] = np.load(data_dir + 'SEQ_' +str(l)  + '/durations_out_' + str(l) + '.npy')
    
n_durations = 9
n_notes = 42

# Training a model to generate music (loop)

In [9]:
import os
import pickle
import numpy
from music21 import note, chord, instrument, stream, duration
import time

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
from models.RNNAttention import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network
from models.RNNAttention import create_network, sample_with_temp
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd

#define the grid
melody_lengths = [12, 16, 24] #
use_attention = True # cambiar a False

embed_size_array = [24,36,48]
rnn_units_array = [32,64,128]

# running parameters
additional_epochs = 4

## loop
for melody_length in melody_lengths:
    
    #load data
    notes_in = notes_in_dict[melody_length]
    notes_out = notes_out_dict[melody_length]
    durations_in = durations_in_dict[melody_length]
    durations_out = durations_out_dict[melody_length]
    
    network_input = [notes_in, durations_in]
    network_output = [notes_out, durations_out]

    
    # loop parameters
    for embed_size in embed_size_array:
        for rnn_units in rnn_units_array:
            run_folder = 'run/{}_{}_{}_{}/'.format(melody_length,embed_size,rnn_units,use_attention)
            store_folder = os.path.join(run_folder, 'store')
            weights_folder = os.path.join(run_folder, 'weights')
            
            #define checkpoints
            checkpoint2 = ModelCheckpoint(os.path.join(weights_folder, "weights.h5"),
                            monitor='val_loss',
                            verbose=0,
                            save_best_only=True,
                            mode='min')
            early_stopping = EarlyStopping(monitor='val_loss'
                                , restore_best_weights=True
                                , patience = 5 )
            callbacks_list = [checkpoint2, early_stopping]
            
            #create directories
            if not os.path.exists(run_folder):
                os.mkdir(run_folder)
                os.mkdir(os.path.join(run_folder, 'store'))
                os.mkdir(os.path.join(run_folder, 'output'))
                os.mkdir(os.path.join(run_folder, 'weights'))
                os.mkdir(os.path.join(run_folder, 'viz'))

            #train model to capture validation and n_epochs
            model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)
            trainHistory = model.fit(network_input, network_output
                  , epochs=1000, batch_size=128
                  , validation_split = 0.2
                  , callbacks=callbacks_list
                  , shuffle=True)

            #retrain with all data n_epochs from above 
            model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)
            fullTrainHistory = model.fit(network_input, network_output
                  , epochs=early_stopping.stopped_epoch + additional_epochs, batch_size=128
                  , validation_split = 0
                  , shuffle=False)
            
            #save fullTrained model
            model.save(store_folder + '/model.h5')

            #save training history
            with open(store_folder + '/trainHistory.json', mode='w') as f:
                pd.DataFrame(trainHistory.history).to_json(f)

            with open(store_folder + '/fullTrainHistory.json', mode='w') as f:
                pd.DataFrame(fullTrainHistory.history).to_json(f)

            #write the parameters
            text_file = open(store_folder + "/parameters.txt", "w")
            text_file.write(str(embed_size)+","+str(rnn_units)+","+str(use_attention)+ "," + str(len(notes_in[0])) +"\n")
            text_file.close()

Epoch 1/1000
197/197 [==============================] - 6s 13ms/step - loss: 4.8892 - pitch_loss: 3.4583 - duration_loss: 1.4309 - val_loss: 4.0000 - val_pitch_loss: 3.0707 - val_duration_loss: 0.9293
Epoch 2/1000
197/197 [==============================] - 2s 10ms/step - loss: 4.0645 - pitch_loss: 3.1230 - duration_loss: 0.9415 - val_loss: 3.9331 - val_pitch_loss: 3.0013 - val_duration_loss: 0.9319
Epoch 3/1000
197/197 [==============================] - 2s 10ms/step - loss: 3.9612 - pitch_loss: 3.0385 - duration_loss: 0.9226 - val_loss: 3.7779 - val_pitch_loss: 2.8495 - val_duration_loss: 0.9284
Epoch 4/1000
197/197 [==============================] - 2s 9ms/step - loss: 3.7985 - pitch_loss: 2.8700 - duration_loss: 0.9285 - val_loss: 3.6658 - val_pitch_loss: 2.7411 - val_duration_loss: 0.9247
Epoch 5/1000
197/197 [==============================] - 2s 10ms/step - loss: 3.6705 - pitch_loss: 2.7649 - duration_loss: 0.9056 - val_loss: 3.5859 - val_pitch_loss: 2.6674 - val_duration_loss: 0.9

197/197 [==============================] - 2s 10ms/step - loss: 2.9147 - pitch_loss: 2.0811 - duration_loss: 0.8336 - val_loss: 2.9692 - val_pitch_loss: 2.1053 - val_duration_loss: 0.8640
Epoch 42/1000
197/197 [==============================] - 2s 12ms/step - loss: 2.9122 - pitch_loss: 2.0723 - duration_loss: 0.8399 - val_loss: 2.9919 - val_pitch_loss: 2.1225 - val_duration_loss: 0.8694
Epoch 43/1000
197/197 [==============================] - 2s 10ms/step - loss: 2.8916 - pitch_loss: 2.0648 - duration_loss: 0.8269 - val_loss: 2.9648 - val_pitch_loss: 2.1065 - val_duration_loss: 0.8583
Epoch 44/1000
197/197 [==============================] - 2s 10ms/step - loss: 2.8760 - pitch_loss: 2.0468 - duration_loss: 0.8292 - val_loss: 2.9630 - val_pitch_loss: 2.1041 - val_duration_loss: 0.8589
Epoch 45/1000
197/197 [==============================] - 2s 10ms/step - loss: 2.8794 - pitch_loss: 2.0551 - duration_loss: 0.8243 - val_loss: 2.9591 - val_pitch_loss: 2.1007 - val_duration_loss: 0.8584
Epoc

246/246 [==============================] - 3s 11ms/step - loss: 3.0615 - pitch_loss: 2.2000 - duration_loss: 0.8615
Epoch 31/65
246/246 [==============================] - 2s 10ms/step - loss: 3.0475 - pitch_loss: 2.1882 - duration_loss: 0.8593
Epoch 32/65
246/246 [==============================] - 2s 9ms/step - loss: 3.0343 - pitch_loss: 2.1772 - duration_loss: 0.8571
Epoch 33/65
246/246 [==============================] - 2s 9ms/step - loss: 3.0224 - pitch_loss: 2.1672 - duration_loss: 0.8551
Epoch 34/65
246/246 [==============================] - 2s 9ms/step - loss: 3.0114 - pitch_loss: 2.1582 - duration_loss: 0.8532
Epoch 35/65
246/246 [==============================] - 2s 9ms/step - loss: 3.0013 - pitch_loss: 2.1498 - duration_loss: 0.8515
Epoch 36/65
246/246 [==============================] - 2s 9ms/step - loss: 2.9918 - pitch_loss: 2.1420 - duration_loss: 0.8498
Epoch 37/65
246/246 [==============================] - 2s 9ms/step - loss: 2.9830 - pitch_loss: 2.1346 - duration_loss: 0

Epoch 19/1000
197/197 [==============================] - 3s 15ms/step - loss: 2.9414 - pitch_loss: 2.0969 - duration_loss: 0.8445 - val_loss: 3.0180 - val_pitch_loss: 2.1401 - val_duration_loss: 0.8779
Epoch 20/1000
197/197 [==============================] - 3s 16ms/step - loss: 2.9193 - pitch_loss: 2.0798 - duration_loss: 0.8395 - val_loss: 2.9705 - val_pitch_loss: 2.1027 - val_duration_loss: 0.8679
Epoch 21/1000
197/197 [==============================] - 3s 15ms/step - loss: 2.9016 - pitch_loss: 2.0625 - duration_loss: 0.8391 - val_loss: 2.9621 - val_pitch_loss: 2.0909 - val_duration_loss: 0.8711
Epoch 22/1000
197/197 [==============================] - 3s 16ms/step - loss: 2.8885 - pitch_loss: 2.0473 - duration_loss: 0.8411 - val_loss: 2.9753 - val_pitch_loss: 2.1114 - val_duration_loss: 0.8639
Epoch 23/1000
197/197 [==============================] - 3s 15ms/step - loss: 2.8771 - pitch_loss: 2.0394 - duration_loss: 0.8377 - val_loss: 2.9395 - val_pitch_loss: 2.0740 - val_duration_los

KeyboardInterrupt: 